<a href="https://colab.research.google.com/github/Abhish2702/BTP-Depression-Detection-/blob/main/btp_using_fasttext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
import string
nltk.download('punkt')
nltk.download('stopwords')
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
d_tweets_dataset=pd.read_csv("/content/clean_d_tweets.csv")
non_d_tweets_dataset=pd.read_csv("/content/clean_non_d_tweets.csv")
d_tweets_dataset.tweet = d_tweets_dataset.tweet.fillna('')
non_d_tweets_dataset.tweet = non_d_tweets_dataset.tweet.fillna('')

In [3]:
d_tweets=d_tweets_dataset['tweet']

In [4]:
non_d_tweets=non_d_tweets_dataset['tweet']

In [5]:
y1=np.ones((3082,1),dtype="int")

In [6]:
y2=np.zeros((2505,1),dtype="int")

In [7]:
y=np.concatenate((y1,y2),axis=0)

In [8]:
res=pd.DataFrame(y)

In [9]:
txt=pd.concat([d_tweets,non_d_tweets],axis=0)

In [10]:
txt.shape
txt=txt.reset_index(drop="true")

In [11]:
df=pd.concat([txt,res],axis=1,join="inner")

In [12]:
df.columns=["text","result"]


In [13]:
import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [25]:
tokenized_text = [word_tokenize(text.lower()) for text in df['text']]

AttributeError: ignored

In [15]:
from gensim.models import FastText

# Train FastText model
fasttext_model = FastText(sentences=tokenized_text, vector_size=100, window=5, min_count=1, workers=4, sg=1)


In [26]:
# Get the FastText word vectors
word_vectors = fasttext_model.wv


In [33]:
# Convert text to FastText word embeddings
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_text)
X_sequences = tokenizer.texts_to_sequences(tokenized_text)
X_padded = pad_sequences(X_sequences, maxlen=100)

# X_fasttext_padded = pad_sequences(X_fasttext, maxlen=100)
# X_fasttext_padded
# y = df['result'].values


In [38]:
# Prepare target variable y (same as before)

# Split the data into train and test sets
from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(X_padded, y, test_size=0.2, random_state=42)


In [37]:
X_fasttext = []
for sentence in tokenized_text:
    embedding = []
    for word in sentence:
        if word in word_vectors:
            embedding.append(word_vectors[word])
    X_fasttext.append(embedding)

# Find the maximum sequence length in X_fasttext
max_sequence_length = max(len(embedding) for embedding in X_fasttext)

# Pad sequences to the maximum length
X_fasttext_padded = pad_sequences(X_fasttext, maxlen=100, dtype='float32', padding='post')
X_fasttext_padded.shape
y = df['result'].values

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X_fasttext_padded, y, test_size=0.2, random_state=42)

In [42]:
# Define the LSTM-RNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
model = Sequential()
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(100, 100)))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 7: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 8: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Step 9: Make Predictions (same as before)


Epoch 1/10
140/140 [==============================] - 24s 154ms/step - loss: 0.6883 - accuracy: 0.5525 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 2/10
140/140 [==============================] - 21s 150ms/step - loss: 0.6887 - accuracy: 0.5547 - val_loss: 0.6894 - val_accuracy: 0.5456
Epoch 3/10
140/140 [==============================] - 21s 150ms/step - loss: 0.6878 - accuracy: 0.5531 - val_loss: 0.6890 - val_accuracy: 0.5456
Epoch 4/10
140/140 [==============================] - 22s 155ms/step - loss: 0.6865 - accuracy: 0.5572 - val_loss: 0.6878 - val_accuracy: 0.5519
Epoch 5/10
140/140 [==============================] - 21s 151ms/step - loss: 0.6746 - accuracy: 0.5831 - val_loss: 0.6740 - val_accuracy: 0.6109
Epoch 6/10
140/140 [==============================] - 21s 151ms/step - loss: 0.5918 - accuracy: 0.6811 - val_loss: 0.5158 - val_accuracy: 0.7585
Epoch 7/10
140/140 [==============================] - 21s 150ms/step - loss: 0.5542 - accuracy: 0.7308 - val_loss: 0.5385 - val_ac

Using Glove

In [43]:
import spacy

# Load pre-trained GloVe embeddings using spaCy (make sure to download the appropriate GloVe model)
nlp = spacy.load("en_core_web_md")
word_vectors = nlp.vocab

# Get word vectors from GloVe
def get_word_vector(word):
    if word in word_vectors:
        return word_vectors[word].vector
    else:
        return None


OSError: ignored

In [44]:
!pip install spacy

In [ ]:
!python -m spacy download en_core_web_md

In [46]:
import spacy

# Load pre-trained GloVe embeddings using spaCy (make sure to download the appropriate GloVe model)
nlp = spacy.load("en_core_web_md")
word_vectors = nlp.vocab

# Get word vectors from GloVe
def get_word_vector(word):
    if word in word_vectors:
        return word_vectors[word].vector
    else:
        return None


In [47]:
# Convert text to GloVe word embeddings
X_glove = []
for sentence in tokenized_text:
    embedding = [get_word_vector(word) for word in sentence]
    embedding = [word_embedding for word_embedding in embedding if word_embedding is not None]
    X_glove.append(embedding)

# Find the maximum sequence length in X_glove
max_sequence_length = max(len(embedding) for embedding in X_glove)

# Pad sequences to the maximum length
X_glove_padded = pad_sequences(X_glove, maxlen=max_sequence_length, dtype='float32', padding='post')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_glove_padded, y, test_size=0.2, random_state=42)


In [48]:
# Define the LSTM-RNN Model
model = Sequential()
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2, input_shape=(max_sequence_length, 300)))  # 300 is the dimensionality of GloVe vectors
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 4: Train the Model
model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Step 5: Evaluate the Model
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

# # Step 6: Make Predictions
# predictions = model.predict_classes(X_test)


Epoch 1/10
140/140 [==============================] - 10s 56ms/step - loss: 0.5944 - accuracy: 0.6785 - val_loss: 0.5373 - val_accuracy: 0.7191
Epoch 2/10
140/140 [==============================] - 7s 47ms/step - loss: 0.5610 - accuracy: 0.7080 - val_loss: 0.5421 - val_accuracy: 0.7120
Epoch 3/10
140/140 [==============================] - 7s 53ms/step - loss: 0.5524 - accuracy: 0.7169 - val_loss: 0.5271 - val_accuracy: 0.7236
Epoch 4/10
140/140 [==============================] - 7s 54ms/step - loss: 0.5484 - accuracy: 0.7122 - val_loss: 0.5410 - val_accuracy: 0.7299
Epoch 5/10
140/140 [==============================] - 7s 47ms/step - loss: 0.5399 - accuracy: 0.7131 - val_loss: 0.5289 - val_accuracy: 0.7200
Epoch 6/10
140/140 [==============================] - 7s 53ms/step - loss: 0.5338 - accuracy: 0.7299 - val_loss: 0.5379 - val_accuracy: 0.7200
Epoch 7/10
140/140 [==============================] - 7s 47ms/step - loss: 0.5308 - accuracy: 0.7234 - val_loss: 0.5406 - val_accuracy: 0.718